#Default Signages

In [ ]:
import pandas as pd

def generate_eoa_signages(data_list):
    frames = []
    for data in data_list:
        start_aisle = data['start_aisle']
        end_aisle = data['end_aisle']
        section = data['section']
        range_segments = data['range'].split(',')

        # Generate a list of aisle numbers
        aisles = list(range(start_aisle, end_aisle + 1))

        for range_ in range_segments:
            range_ = range_.strip()  # Remove leading/trailing whitespace

            # Determine pairs and directions for low and high ends
            high_end_pairs = [(aisles[i], aisles[i+1]) for i in range(0, len(aisles) - 1, 2)]
            low_end_pairs = [(aisles[i+1], aisles[i]) for i in range(0, len(aisles) - 1, 2)]

            # Create DataFrame for Low End Signages
            low_end_df = pd.DataFrame(low_end_pairs, columns=['First Aisle', 'Second Aisle'])
            low_end_df['Direction'] = 'Low End of Aisle'
            low_end_df['Range'] = range_
            low_end_df['Section'] = section

            # Create DataFrame for High End Signages
            high_end_df = pd.DataFrame(high_end_pairs, columns=['First Aisle', 'Second Aisle'])
            high_end_df['Direction'] = 'High End of Aisle'
            high_end_df['Range'] = f"{range_.split('-')[1]}-{range_.split('-')[0]}"
            high_end_df['Section'] = section

            # Combine both DataFrames
            full_df = pd.concat([low_end_df, high_end_df], ignore_index=True)

            # Add to list of all frames
            frames.append(full_df)

    # Concatenate all dataframes into one
    signages_df = pd.concat(frames, ignore_index=True)

    # Format the DataFrame into the required signage format
    signages_df['EOA Signage'] = signages_df.apply(
        lambda x: f"{x['Section']}\t{x['Second Aisle']}\t{x['Range']}\t{x['Section']}\t{x['First Aisle']}\t{x['Range']}\t{x['Direction']} {x['Second Aisle']}/{x['First Aisle']}",
        axis=1
    )

    return signages_df


#For Overlapping Aisles across mods

In [ ]:
def handle_overlap(df, sec1, aisle1, range1, sec2, aisle2, range2):
    # Split ranges and trim whitespace
    range1_list = [r.strip() for r in range1.split(',')]
    range2_list = [r.strip() for r in range2.split(',')]

    # Create overlap signage for each pair of ranges
    overlap_data = []
    for r1, r2 in zip(range1_list, range2_list):
        # Flip the range for High End signages|
        r1_flipped = '-'.join(r1.split('-')[::-1])
        r2_flipped = '-'.join(r2.split('-')[::-1])

        # Low End of Aisle
        overlap_data.append({
            'Section1': sec1,
            'Aisle1': aisle1,
            'Range1': r1,
            'Section2': sec2,
            'Aisle2': aisle2,
            'Range2': r2,
            'Direction': f'Low End of Aisle {aisle2}/{aisle1}'
        })
        # High End of Aisle
        overlap_data.append({
            'Section1': sec2,
            'Aisle1': aisle2,
            'Range1': r2_flipped,
            'Section2': sec1,
            'Aisle2': aisle1,
            'Range2': r1_flipped,
            'Direction': f'High End of Aisle {aisle1}/{aisle2}'
        })

    # Append the overlap data to the DataFrame
    overlap_df = pd.DataFrame(overlap_data)
    overlap_df['EOA Signage'] = overlap_df.apply(
        lambda x: f"{x['Section1']}\t{x['Aisle1']}\t{x['Range1']}\t{x['Section2']}\t{x['Aisle2']}\t{x['Range2']}\t{x['Direction']}",
        axis=1
    )

    # Append the overlap rows to the end of the DataFrame
    df = pd.concat([df, overlap_df]).reset_index(drop=True)

    return df



#For individual Aisles

In [ ]:
def individual_aisle(df, section1, section2, aisle, ranges):
    # Split ranges and trim whitespace
    range_list = [r.strip() for r in ranges.split(',')]

    # Create signage data for both low end and high end
    signage_data = []
    for range_ in range_list:
        # Low End of Aisle
        if section1 == "Think Safety":
            signage_low = {
                'Section1': 'Think Safety',
                'Aisle1': '',
                'Range1': '',
                'Section2': section2,
                'Aisle2': aisle,
                'Range2': range_,
                'Direction': f'Low End of Aisle {aisle}/'
            }
            range_flipped = '-'.join(range_.split('-')[::-1])
            signage_high = {
                'Section1': section2,
                'Aisle1': aisle,
                'Range1': range_flipped,
                'Section2': 'Think Safety',
                'Aisle2': '',
                'Range2': '',
                'Direction': f'High End of Aisle /{aisle}'
            }
        else:
            signage_low = {
                'Section1': section1,
                'Aisle1': aisle,
                'Range1': range_,
                'Section2': 'Think Safety',
                'Aisle2': '',
                'Range2': '',
                'Direction': f'Low End of Aisle {aisle}/'
            }
            # High End of Aisle - reverse the range for the high end
            range_flipped = '-'.join(range_.split('-')[::-1])
            signage_high = {
                'Section1': 'Think Safety',
                'Aisle1': '',
                'Range1': '',
                'Section2': section1,
                'Aisle2': aisle,
                'Range2': range_flipped,
                'Direction': f'High End of Aisle /{aisle}'
            }

        signage_data.extend([signage_low, signage_high])

    # Convert list of dicts to DataFrame
    signage_df = pd.DataFrame(signage_data)
    signage_df['EOA Signage'] = signage_df.apply(
        lambda x: f"{x['Section1']}\t{x['Aisle1']}\t{x['Range1']}\t{x['Section2']}\t{x['Aisle2']}\t{x['Range2']}\t{x['Direction']}",
        axis=1
    )

    # Append the individual aisle signage to the existing DataFrame
    return pd.concat([df, signage_df]).reset_index(drop=True)



#Inputs


In [ ]:
#default signs
data_list = [
    {'start_aisle': 213, 'end_aisle': 262, 'section': 'P-2-A', 'range': '200-290'},
    {'start_aisle': 201, 'end_aisle': 262, 'section': 'P-2-A', 'range': '340-430, 460-550, 580-660'},
    {'start_aisle': 201, 'end_aisle': 238, 'section': 'P-2-B', 'range': '200-290, 340-430, 460-550, 580-660'},
    {'start_aisle': 201, 'end_aisle': 224, 'section': 'P-2-C', 'range': '200-290, 340-430, 460-550, 580-660'},
    {'start_aisle': 201, 'end_aisle': 244, 'section': 'P-2-D', 'range': '200-240, 260-310, 330-370, 390-430'}
]

df = generate_eoa_signages(data_list)

#overlap

df = handle_overlap(df, 'P-2-C', 225, '200-290, 340-430, 460-550, 580-660', 'P-2-D', 200, '200-240, 260-310, 330-370, 390-430')

# individual aisles
df = individual_aisle(df, 'Think Safety', 'P-2-A', 212, '200-290')
df = individual_aisle(df, 'Think Safety', 'P-2-A', 200, '340-430, 460-550, 580-660')
df = individual_aisle(df, 'P-2-D', 'Think Safety', 245, '190-430')



In [ ]:

def print_signages(df):
    print("Index\tEOA Signage")
    for index, row in df.iterrows():
        print(f"{index}\t{row['EOA Signage']}")

# Example call to print the DataFrame
print_signages(df)

Index	EOA Signage
0	P-2-A	213	200-290	P-2-A	214	200-290	Low End of Aisle 213/214
1	P-2-A	215	200-290	P-2-A	216	200-290	Low End of Aisle 215/216
2	P-2-A	217	200-290	P-2-A	218	200-290	Low End of Aisle 217/218
3	P-2-A	219	200-290	P-2-A	220	200-290	Low End of Aisle 219/220
4	P-2-A	221	200-290	P-2-A	222	200-290	Low End of Aisle 221/222
5	P-2-A	223	200-290	P-2-A	224	200-290	Low End of Aisle 223/224
6	P-2-A	225	200-290	P-2-A	226	200-290	Low End of Aisle 225/226
7	P-2-A	227	200-290	P-2-A	228	200-290	Low End of Aisle 227/228
8	P-2-A	229	200-290	P-2-A	230	200-290	Low End of Aisle 229/230
9	P-2-A	231	200-290	P-2-A	232	200-290	Low End of Aisle 231/232
10	P-2-A	233	200-290	P-2-A	234	200-290	Low End of Aisle 233/234
11	P-2-A	235	200-290	P-2-A	236	200-290	Low End of Aisle 235/236
12	P-2-A	237	200-290	P-2-A	238	200-290	Low End of Aisle 237/238
13	P-2-A	239	200-290	P-2-A	240	200-290	Low End of Aisle 239/240
14	P-2-A	241	200-290	P-2-A	242	200-290	Low End of Aisle 241/242
15	P-2-A	243	200-290	P-2-A	244	2